In [1]:
API_KEY = ""

In [2]:
#Task 1: Fetch Basic Stock Data
import requests

def get_stock_data():
    """Fetch stock data for Microsoft, Apple, and NVIDIA."""
    symbols = ["MSFT", "AAPL", "NVDA"]  # Predefined list
    url = "https://yfapi.net/v6/finance/quote"
    headers = {"x-api-key": API_KEY}
    params = {"symbols": ",".join(symbols)}

    response = requests.get(url, headers=headers, params=params).json()

    for stock in response["quoteResponse"]["result"]:
        print(f"Stock Ticker: {stock['symbol']}, "
              f"Company: {stock.get('shortName', 'N/A')}, "
              f"Current Market Price: ${stock.get('regularMarketPrice', 'N/A'):.2f}")

# Run the function
get_stock_data()

Stock Ticker: MSFT, Company: Microsoft Corporation, Current Market Price: $404.00
Stock Ticker: AAPL, Company: Apple Inc., Current Market Price: $247.10
Stock Ticker: NVDA, Company: NVIDIA Corporation, Current Market Price: $130.28


In [6]:
#Task 2.1: Fetch Additional Data Using Modules Dynamically
import pandas as pd
import requests

def get_additional_data():
    "Fetch stock data based on the user's chosen module, convert to DataFrame, and display."
    symbols = ["MSFT", "AAPL", "NVDA"]  # Fixed stocks
    module_choice = input("Choose a module (summaryDetail for 52W High/Low, financialData for ROA): ").strip()

    stock_data = []  # List to store stock details

    for symbol in symbols:
        url = f"https://yfapi.net/v11/finance/quoteSummary/{symbol}"
        headers = {"x-api-key": API_KEY}
        params = {"modules": module_choice}

        response = requests.get(url, headers=headers, params=params).json()

        result = response["quoteSummary"]["result"][0].get(module_choice, {})

        # Extract necessary fields based on the selected module
        if module_choice == "summaryDetail":
            stock_data.append({
                "Stock": symbol,
                "52 Week High": result.get("fiftyTwoWeekHigh", {}).get("raw", "N/A"),
                "52 Week Low": result.get("fiftyTwoWeekLow", {}).get("raw", "N/A")
            })
        elif module_choice == "financialData":
            stock_data.append({
                "Stock": symbol,
                "Return on Assets (ROA)": f"{result.get('returnOnAssets', {}).get('raw', 'N/A'):.2%}"
                if result.get('returnOnAssets', {}).get('raw', "N/A") != "N/A" else "N/A"
            })

    # Convert list to DataFrame and display
    df = pd.DataFrame(stock_data)
    print("\nStock Data:")
    display(df)

# Run the function
get_additional_data()


Choose a module (summaryDetail for 52W High/Low, financialData for ROA): financialData

Stock Data:


,Stock,Return on Assets (ROA)
0,MSFT,14.65%
1,AAPL,22.52%
2,NVDA,59.13%


In [7]:
#Task 2.2: Fetch Additional Data Using Modules
def get_trending_stocks():
    """Fetch trending stocks and display their Ticker, Name, Price, 52 Week High & Low."""
    url = f"https://yfapi.net/v1/finance/trending/US"  # Fetching US trending stocks
    headers = {"x-api-key": API_KEY}

    response = requests.get(url, headers=headers).json()

    trending_symbols = [stock["symbol"] for stock in response["finance"]["result"][0]["quotes"][:5]]  # Top 5
    print(f"Fetching additional details for trending stocks: {', '.join(trending_symbols)}")

    # Fetch additional data (52 Week High/Low & Current Price)
    url = f"https://yfapi.net/v6/finance/quote"
    params = {"symbols": ",".join(trending_symbols)}

    response = requests.get(url, headers=headers, params=params).json()

    # Display details for each stock
    for stock in response["quoteResponse"]["result"]:
        print(f"\nStock: {stock['symbol']} ({stock.get('shortName', 'N/A')})")
        print(f"Current Price: ${stock.get('regularMarketPrice', 'N/A')}")
        print(f"52 Week High: ${stock.get('fiftyTwoWeekHigh', 'N/A')}")
        print(f"Week Low: ${stock.get('fiftyTwoWeekLow', 'N/A')}")

# Run the function
get_trending_stocks()


Fetching additional details for trending stocks: PLTR, HIMS, BTC-USD, TEM, ZM

Stock: PLTR (Palantir Technologies Inc.)
Current Price: $90.68
52 Week High: $125.41
Week Low: $20.33

Stock: HIMS (Hims & Hers Health, Inc.)
Current Price: $51.31
52 Week High: $72.98
Week Low: $9.67

Stock: BTC-USD (Bitcoin USD)
Current Price: $91404.57
52 Week High: $109114.88
Week Low: $49121.24

Stock: TEM (Tempus AI, Inc.)
Current Price: $69.57
52 Week High: $91.45
Week Low: $22.89

Stock: ZM (Zoom Communications, Inc.)
Current Price: $81.1
52 Week High: $92.8
Week Low: $55.06


In [9]:
#BONUS Challenge 1

import pandas as pd
import requests

def format_currency(value):
    """Format large currency values with commas and a dollar sign."""
    if isinstance(value, (int, float)):
        return f"${value:,.2f}"  # Format as "$1,500,000.00"
    return "N/A"

def get_dynamic_module_data():
    """Allow users to fetch data from any module dynamically and format currency values."""
    symbols = ["MSFT", "AAPL", "NVDA"]  # Stocks to check
    module_choice = input("Enter the module name (e.g., summaryDetail, financialData, defaultKeyStatistics): ").strip()

    stock_data = []

    for symbol in symbols:
        url = f"https://yfapi.net/v11/finance/quoteSummary/{symbol}"
        headers = {"x-api-key": API_KEY}
        params = {"modules": module_choice}

        response = requests.get(url, headers=headers, params=params).json()

        result = response["quoteSummary"]["result"][0].get(module_choice, {})

        # Convert all values to strings & format currency where applicable
        formatted_data = {key: format_currency(value.get("raw", "N/A")) if isinstance(value, dict) else value
                          for key, value in result.items()}
        formatted_data["Stock"] = symbol  # Add stock symbol

        stock_data.append(formatted_data)

    # Convert to DataFrame and display
    df = pd.DataFrame(stock_data)
    print("\nStock Data:")
    display(df)

# Run the function
get_dynamic_module_data()


Enter the module name (e.g., summaryDetail, financialData, defaultKeyStatistics): defaultKeyStatistics

Stock Data:


,maxAge,priceHint,enterpriseValue,forwardPE,profitMargins,floatShares,sharesOutstanding,sharesShort,sharesShortPriorMonth,sharesShortPreviousMonthDate,...,lastCapGain,annualHoldingsTurnover,latestFundingDate,latestAmountRaised,latestImpliedValuation,latestShareClass,leadInvestor,fundingToDate,totalFundingRounds,Stock
0,1,$2.00,"$3,034,682,753,024.00",$26.85,$0.35,"$7,423,723,340.00","$7,433,979,904.00","$62,492,832.00","$62,178,630.00","$1,735,603,200.00",...,N/A,N/A,N/A,N/A,N/A,None,None,N/A,N/A,MSFT
1,1,$2.00,"$3,756,074,729,472.00",$29.99,$0.24,"$14,998,187,904.00","$15,022,100,480.00","$124,917,523.00","$157,008,120.00","$1,735,603,200.00",...,N/A,N/A,N/A,N/A,N/A,None,None,N/A,N/A,AAPL
2,1,$2.00,"$3,162,295,238,656.00",$29.32,$0.56,"$23,504,522,400.00","$24,490,000,384.00","$271,436,938.00","$287,895,394.00","$1,735,603,200.00",...,N/A,N/A,N/A,N/A,N/A,None,None,N/A,N/A,NVDA
